In [4]:
from src.utils.hybridsearch import run_hybrid_search
res = run_hybrid_search(
    query="có ip 15 128 ko bro",
    k=20,
    is_bm25_enable=True
)
res

NEO4J_URI: bolt://neo4j:7687
USERNAME: neo4j
DATABASE: neo4j


ValueError: Cannot resolve address neo4j:7687

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


client = MultiServerMCPClient({
    "hybridsearch": {
        "url": "http://localhost:8000/mcp", 
        "transport": "streamable_http"
    }
})
async with client.session("hybridsearch") as session:
    tools = await load_mcp_tools(session)
    print(tools)
    # call tool
    results = await session.call_tool("hybrid_search",
    {
        "query": "có ip ko",
        "k": 30
    })
    print(results)

[StructuredTool(name='hybrid_search', description="Retrieve restaurant and dish info from the knowledge base.\nSupports:\n- Restaurant details: name, location, cuisine, opening hours\n- Dish details: name, ingredients, dietary info, cooking style\nIncreasing 'k' allows fetching more candidate results for better coverage.", args_schema={'properties': {'query': {'description': 'Restaurant or dish name, location, or type', 'maxLength': 300, 'minLength': 3, 'title': 'Query', 'type': 'string'}, 'k': {'default': 30, 'description': 'Number of top results to retrieve: 30=fast, 50=broad', 'enum': [30, 50], 'title': 'K', 'type': 'integer'}, 'cuisine': {'default': '', 'description': 'Optional cuisine type to filter results', 'maxLength': 50, 'title': 'Cuisine', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x79a3fc41bf60>)]
meta=None content=[TextContent(type='text'

In [8]:
import pandas as pd
file_path="src\data\hoanghamobile_with_summary.csv"
df = pd.read_csv(file_path, nrows=1, encoding="utf-8")
all_columns = df.columns.tolist()
# Define which column will be the content
content_column = "combined_info"
# All other columns go into metadata (exclude content_column)
metadata_columns = [col for col in all_columns if col != content_column]

from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(file_path,
    csv_args={
        'delimiter': ',',
        'quotechar': '"'
    },
    content_columns=content_column,
    metadata_columns=metadata_columns,
    encoding='utf-8'
)
documents = loader.load()